Common Code:

In [1]:
from pyspark.sql import SparkSession
import re
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext

Usersxml = sc.textFile("Users.xml")
PostHistoryxml = sc.textFile("PostHistory.xml")
Commentsxml = sc.textFile("Comments.xml")

def valuesForKey(key,line):
  pattern = ' '+key+'=\\"(.*?)\\"'
  xmlpattern=re.compile(pattern)
  print(xmlpattern)
  return(xmlpattern.findall(line))

#### Question 1: From the Users.xml file, find all users which are from Georgia and output to screen their DisplayName only.

In [2]:
reGeorgia=re.compile('Location=\".*Georgia.*?\"')
#filtering rows with location Georgia -> extracting DisplayName -> formatting -> removing null DisplayNames
Georgia_DN_RDD=Usersxml.filter(lambda line: reGeorgia.findall(line)).map(lambda line: valuesForKey('DisplayName',line)).map(lambda x: x[0]).filter(lambda name: len(name)>0)
print(Georgia_DN_RDD.collect())

['Nick Larsen', 'Ayush', 'ontek', 'vkb', 'Giorgi Gzirishvili', 'Rafael_Espericueta', 'Alex Azazel', 'James Wang', 'Dan Anton', 'ode2k', 'Manish Ranjan', 'Dato Janez', 'Neuromeda', 'Mr. Rooter of Southeast GA', 'Abesalomi Gogatishvili', 'FranGoitia', 'Jenna Kwon', 'Tedo G.', 'mlibre', 'Jeff', 'Aditya Gogoi', 'Jesse Scherer', 'Melinda Weathers', 'N.C.W.', 'DRozen', 'rajb245', 'G.T.', 'Dato Gogolashvili', 'Ankush Chauhan', 'Scott', 'David Jashi', 'Ilia Chigogidze', 'German', 'Franco Piccolo', 'Dean Webb', 'Tae-Sung Shin', 'Clark Lin', 'Adam Sarwar', 'Gio Nari', 'mLstudent33', 'Lance Ruo Zhang', 'alerera', 'Max Boonbandansook', 'ErinPac']


#### Question 2: Using the Users.xml file, provide the count for all users which joined (CreationDate) in 2017. 

In [3]:
CD2017=re.compile('CreationDate=\"2017\-')
#filtering rows with CD 2017 -> extracting Displat Name (DN) -> formatting -> removing null data -> Displaying
CD2017_RDD=Usersxml.filter(lambda line: CD2017.findall(line)).map(lambda line: valuesForKey('DisplayName',line)).map(lambda x: x[0]).filter(lambda name: len(name)>0)
print('Count of users joined in 2017: '+str(CD2017_RDD.count()))

Count of users joined in 2017: 14239


#### Question 3: Using the PostHistory file, count the number of Posts that feature the words “Spark” and “Scala”. Output this to the screen.

In [4]:
wSparki=re.compile('Text=\"(.*Spark.*)\"',re.IGNORECASE)
wScalai=re.compile('Text=\"(.*Scala.*)\"',re.IGNORECASE)
wSpark=re.compile('Text=\"(.*Spark.*)\"')
wScala=re.compile('Text=\"(.*Scala.*)\"')

RDD_SparknScala=PostHistoryxml.filter(lambda line: wSpark.findall(line)).filter(lambda line: wScala.findall(line))
print('#Post with Spark and Scala both - case sensitive: '+str(RDD_SparknScala.count()))
                                                                                
RDD_SparknScalai=PostHistoryxml.filter(lambda line: wSparki.findall(line)).filter(lambda line: wScalai.findall(line))
print('#Post with Spark and Scala both - ignorecase: '+str(RDD_SparknScalai.count()))

#Post with Spark and Scala both - case sensitive: 95
#Post with Spark and Scala both - ignorecase: 211


#### Question 4: Using the PostHistory file, provide a total count of the words used by each distinct user. In other words, count all words in all posts for each user and display this to screen. You can only identify users by the UserID (30 points). You get 15 bonus points if you get the actual DisplayName of the user.

In [5]:
#creating key-value pair for UserID and words in Post -> removing null UserId data -> formatting -> adding #words for each UserId
User_text_map=PostHistoryxml.map(lambda line: (valuesForKey('UserId',line) ,int(len(str(valuesForKey('Text',line)).split())))).filter(lambda userid: len(userid[0])>0).map(lambda x: (int(x[0][0]),int(x[1]))).reduceByKey(lambda x, y: x + y)

#creating key-value pair for UserID and DisplayName -> removing null UserId data -> formatting -> removing UserId redundancy if any
Id_DN_map=Usersxml.map(lambda line: ((valuesForKey('Id',line),valuesForKey('DisplayName',line)))).filter(lambda x: len(x[0])>0).map(lambda x: (int(x[0][0]),x[1][0])).reduceByKey(lambda x,y: x+y)

#joining UserId, DisplayName and #words in post -> replacing null DisplayName to NoDisplayName
ID_DN_TextC=User_text_map.leftOuterJoin(Id_DN_map).map(lambda x: (x[0],'NoDisplayName' if x[1][1]==None else x[1][1],x[1][0]))

#Sort
ID_DN_TextC_Sorted_RDD=ID_DN_TextC.sortBy(lambda x:x[2],ascending=False)
print('UserID, DisplayName, #words in post')
print(ID_DN_TextC_Sorted_RDD.collect())

UserID, DisplayName, #words in post
[(836, 'Neil Slater', 346877), (67328, 'Esmailian', 193951), (29575, 'Stephen Rauch', 192991), (45264, 'n1k31t4', 142189), (-1, 'Community', 135602), (28175, 'Vaalizaadeh', 109172), (43077, 'Kari', 103725), (29587, 'JahKnows', 81814), (29169, 'Ethan', 66310), (11097, 'Dawny33', 58990), (8820, 'Martin Thoma', 54990), (40853, 'Toros91', 48806), (381, 'Emre', 42174), (8878, 'Kasra Manshaei', 41997), (71219, 'aranglol', 36709), (924, 'Anony-Mousse', 36441), (52291, 'Simon Larsson', 33818), (4683, 'David Marx', 33799), (14904, 'Jan van der Vegt', 33707), (23305, 'oW_', 32540), (71442, 'Peter', 32069), (35644, 'Aditya', 32040), (44456, 'TwinPenguins', 30073), (9420, 'AN6U5', 29759), (1330, 'Brian Spiering', 28831), (8560, 'D.W.', 28662), (16853, 'Ricardo Cruz', 27400), (201, 'Tasos', 27319), (26562, 'timleathart', 26184), (2452, 'Aleksandr Blekh', 25706), (74421, 'Fatemeh Asgarinejad', 25657), (108, 'rapaio', 24061), (38892, 'Siong Thye Goh', 23839), (7121

#### Question 5: Using the users.xml, comments.xml and PostHistory.xml files, produce a single file that includes the following information: DisplayName, Number of Comments, total Score and Number of posts. This file should have the users (DisplayName) sorted by score, descending from higher to lower.

In [6]:
#Collecting UserId and DisplayName from Usersxml -> removing null UserIds -> formatting -> removing UserId redundancy if any
UserId_DN_RDD=Usersxml.map(lambda line: ((valuesForKey('Id',line),valuesForKey('DisplayName',line)))).filter(lambda x: len(x[0])>0).map(lambda x: (x[0][0],x[1][0])).reduceByKey(lambda x,y: x+y)

#Collecting UserId from Commentsxml -> appending tuple with 1 -> removing null UserIds -> formatting -> adding number of comments for each UserId
UserId_Comments_map=Commentsxml.map(lambda line: (valuesForKey('UserId',line))).map(lambda x:(x,1)).filter(lambda userid: len(userid[0])>0).map(lambda x: (x[0][0],x[1])).reduceByKey(lambda x,y:x+y)

#Collecting UserId and Score from Commentsxml -> removing null UserIds -> formatting -> adding Score for each UserId
UserId_Score_RDD=Commentsxml.map(lambda line: (valuesForKey('UserId',line),valuesForKey('Score',line))).filter(lambda x: len(x[0])>0).map(lambda x: (x[0][0],int(x[1][0])) ).reduceByKey(lambda x,y: x+y)
                                 
#Collecting UserId from PostHistoryxml -> appending tuple with 1 -> removing null UserIds -> formatting -> adding number of post for each UserId
UserId_Post_RDD=PostHistoryxml.map(lambda line: (valuesForKey('UserId',line))).map(lambda x:(x,1)).filter(lambda userid: len(userid[0])>0).map(lambda x: (x[0][0],x[1])).reduceByKey(lambda x,y:x+y)


#JOINS
UserId_DN_Comments_RDD=UserId_DN_RDD.leftOuterJoin(UserId_Comments_map)

UserId_DN_Comments_Score_RDD=UserId_DN_Comments_RDD.leftOuterJoin(UserId_Score_RDD)

UserId_DN_Comments_Score_Post_RDD=UserId_DN_Comments_Score_RDD.leftOuterJoin(UserId_Post_RDD)

#formatting and removing UserId
DN_Comments_Score_Post_RDD2=UserId_DN_Comments_Score_Post_RDD.map(lambda x: (x[1][0][0][0],0 if x[1][0][0][1]==None else x[1][0][0][1],0 if x[1][0][1]==None else x[1][0][1],0 if x[1][1]==None else x[1][1]) )


#Sorting by Score
DN_Comments_Score_Post_Sorted_RDD=DN_Comments_Score_Post_RDD2.sortBy(lambda x: x[2],ascending=False)

file = open('data.txt', 'w')
for i in DN_Comments_Score_Post_Sorted_RDD.take(DN_Comments_Score_Post_Sorted_RDD.count()) :
    file.write(str(i))
    
